In [116]:
import math
from copy import deepcopy
from itertools import product

from rdkit import Chem, DataStructs

import sqlite3
import pandas as pd

import numpy as np
import os
import glob

In [117]:
def read_db_data_in_folder(folder_path: str) -> pd.DataFrame:
    """
    Reads all data from the `results` table of all SQLite databases in a specified folder.

    Parameters
    ----------
    folder_path: str
        Path to the folder containing sqlite3 database files.

    Returns
    -------
    pd.DataFrame
        A combined dataframe containing rows from the `results` table 
        from all found databases in the folder. Returns an empty dataframe if none of the 
        tables exist or other errors.
    """
    # Step 1: Identify all .db files in the directory
    db_files = glob.glob(os.path.join(folder_path, "*.db"))

    combined_data = []
    columns = None

    for db_path in db_files:
        conn = sqlite3.connect(db_path)
        
        try:
            df = pd.read_sql("SELECT * FROM results", conn)
            
            if columns is None:
                columns = df.columns.tolist()
            
            combined_data.append(df)
        except sqlite3.OperationalError as e:
            if "no such table: results" in str(e):
                print(f"The table 'results' does not exist in the database at path: {db_path}")
            else:
                raise e
        finally:
            conn.close()

    if not combined_data:
        return pd.DataFrame(columns=columns or [])
    
    return pd.concat(combined_data, ignore_index=True)


In [118]:
df = read_db_data_in_folder("logs/mol_eval_seh_frag/final")
len(df)

1024

In [119]:
# Drop duplicates in the SMILES column
df = df.drop_duplicates(subset=["smi"])

df["mol"] = df["smi"].apply(lambda x: Chem.MolFromSmiles(x))
#df["fp"] = df["smi"].apply(lambda x: Chem.RDKFingerprint(Chem.MolFromSmiles(x)))
df

,smi,r,fr_0,ci_beta,mol
0,CC(C)=CC(C)(C)C,0.036428,0.036428,1.0,<rdkit.Chem.rdchem.Mol object at 0x7f59338775f0>
1,C[SH+]N1CCN(n2cnc3c(-c4c[nH]c(N5C=CCC([n+]6ccc...,0.621887,0.621887,1.0,<rdkit.Chem.rdchem.Mol object at 0x7f593394af20>
2,CC(=O)Nc1ccc[n+](-n2cc([N+](=O)[O-])c(=O)[nH]c...,0.419236,0.419236,1.0,<rdkit.Chem.rdchem.Mol object at 0x7f593394af90>
3,C1CCC(C2CC2)C1,0.103607,0.103607,1.0,<rdkit.Chem.rdchem.Mol object at 0x7f593374bf20>
4,CC(C)C=CC1OC(Cl)C(n2cnc3c(=O)[nH]c([SH](=O)=O)...,0.391219,0.391219,1.0,<rdkit.Chem.rdchem.Mol object at 0x7f593374bf90>
...,...,...,...,...,...
1019,CC(C)(O)n1ccc([PH](=O)[O-])nc1=O,0.283848,0.283848,1.0,<rdkit.Chem.rdchem.Mol object at 0x7f5933765e40>
1020,CC(C)=CCS,0.005097,0.005097,1.0,<rdkit.Chem.rdchem.Mol object at 0x7f5933765eb0>
1021,CC(=CI)C(C)(C)O,0.029174,0.029174,1.0,<rdkit.Chem.rdchem.Mol object at 0x7f5933765f20>
1022,CN(C)C1=CN(c2nc(C(=N)N)cs2)C=CC1,0.360939,0.360939,1.0,<rdkit.Chem.rdchem.Mol object at 0x7f5933765f90>


In [120]:
def compute_diverse_topk(candidates, k=100, tanimoto_thresh=0.7):
    modes = [candidates[0]]
    mode_fps = [Chem.RDKFingerprint(candidates[0][1])]
    for i in range(1, len(candidates)):
        fp = Chem.RDKFingerprint(candidates[i][1])
        sim = DataStructs.BulkTanimotoSimilarity(fp, mode_fps) 
        # if sim to any of the modes is less than thresh, add to modes
        if max(sim) < tanimoto_thresh:
            modes.append(candidates[i])
            mode_fps.append(fp)
        if len(modes) >= k:
            # last_idx = i
            break
    avg_reward_in_topk_modes = np.mean([i[0] for i in modes])  
    return avg_reward_in_topk_modes 

In [121]:
def compute_num_of_modes(candidates, reward_thresh=8, tanimoto_thresh=0.7):
    candidates = sorted(candidates, key=lambda m: m[0], reverse=True)
    # cut of candidates with reward less than reward_thresh
    candidates = [c for c in candidates if c[0] >= reward_thresh]

    num_candidates_above_reward_thresh = len(candidates)
    if num_candidates_above_reward_thresh == 0:
        return 0, 0
    
    modes = [candidates[0]]
    mode_fps = [Chem.RDKFingerprint(candidates[0][1])]
    for i in range(1, len(candidates)):
        fp = Chem.RDKFingerprint(candidates[i][1])
        sim = DataStructs.BulkTanimotoSimilarity(fp, mode_fps) 
        # if sim to any of the modes is less than thresh, add to modes
        if max(sim) < tanimoto_thresh:
            modes.append(candidates[i])
            mode_fps.append(fp)

    num_of_modes = len(modes)
    return num_of_modes,num_candidates_above_reward_thresh

In [122]:
def compute_avg_topk(rewards,k):
    # Sort the rewards
    rewards = sorted(rewards, reverse=True)
    # Get the top k rewards
    topk_rewards = rewards[:k]
    # Return the mean of the top k rewards
    return np.mean(topk_rewards)

In [123]:
def candidates_eval(path, k=100, reward_thresh=8, tanimoto_thresh=0.7):
    df = read_db_data_in_folder(path)

    df = df.drop_duplicates(subset=["smi"])

    if len(df) < k:
        raise ValueError(f"Number of unique SMILES ({len(df)}) is less than k ({k})")

    df["mol"] = df["smi"].apply(lambda x: Chem.MolFromSmiles(x))

    smiles = df["smi"].tolist()
    rewards = df["r"].tolist()
    mols = df["mol"].tolist()

    avg_topk = compute_avg_topk(rewards, k=k)

    candidates = list(zip(rewards,mols))
    candidates = sorted(candidates, key=lambda m: m[0], reverse=True)

    avg_reward_in_topk_modes = compute_diverse_topk(candidates, k=k, tanimoto_thresh=tanimoto_thresh)
    num_of_modes,num_candidates_above_reward_thresh = compute_num_of_modes(candidates, reward_thresh = reward_thresh, tanimoto_thresh=tanimoto_thresh)
    return {"avg_topk": avg_topk, "avg_reward_in_topk_modes": avg_reward_in_topk_modes,"num_of_modes":num_of_modes, "max_reward":max(rewards),"num_candidates":len(candidates),"num_candidates_above_reward_thresh":num_candidates_above_reward_thresh}


In [124]:
path = "logs/mol_eval_seh_frag/final"
k = 100
reward_thresh = 8.0
tanimoto_thresh = 0.7
candidates_eval(path,k,reward_thresh,tanimoto_thresh)

{'avg_topk': 0.5719867277145386,
 'avg_reward_in_topk_modes': 0.5674423676729202,
 'num_of_modes': 0,
 'max_reward': 0.8409422636032104,
 'num_candidates': 959,
 'num_candidates_above_reward_thresh': 0}

In [132]:
df = read_db_data_in_folder(path)

df = df.drop_duplicates(subset=["smi"])

if len(df) < k:
    raise ValueError(f"Number of unique SMILES ({len(df)}) is less than k ({k})")

df["mol"] = df["smi"].apply(lambda x: Chem.MolFromSmiles(x))

smiles = df["smi"].tolist()
rewards = df["r"].tolist()
mols = df["mol"].tolist()


candidates = list(zip(rewards,mols))
candidates = sorted(candidates, key=lambda m: m[0], reverse=True)

avg_reward_in_topk_modes = compute_diverse_topk(candidates, k, tanimoto_thresh)
avg_reward_in_topk_modes

0.5674423676729202

In [136]:
num_of_modes = compute_num_of_modes(candidates, 0.6, 0.7)
num_of_modes

(27, 28)